<a href="https://colab.research.google.com/github/ryunguo/LanguageDetector/blob/main/LanguageDetector.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import pandas as pd
import numpy as np
import sklearn as sk
from sklearn.preprocessing import LabelEncoder
import re
import spacy
from gensim.models import Word2Vec

In [6]:
df = pd.read_csv("Language Detection.csv")

In [7]:
language_list = ['English', 'French', 'Italian', 'Spanish', 'Portugeese']
df = df.loc[df['Language'].isin(language_list)]
df

,Text,Language
0,"Nature, in the broadest sense, is the natural...",English
1,"""Nature"" can refer to the phenomena of the phy...",English
2,"The study of nature is a large, if not the onl...",English
3,"Although humans are part of nature, human acti...",English
4,[1] The word nature is borrowed from the Old F...,English
...,...,...
7807,"qual è stato il tuo errore, ti diamo da mangia...",Italian
7808,narcisa ha cambiato i suoi modi in un primo mo...,Italian
7809,Come' Il narcisismo di adesso Marian ha detto ...,Italian
7810,immagino che non vorrebbe più pane d'oro adess...,Italian


***Data Wrangling***

In [8]:
x = df['Text']
y = df['Language']

In [9]:
le = LabelEncoder()
y=le.fit_transform(y)

In [10]:
data = []

def removeNonsense(text):
  text = re.sub(r'[0-9]', '', text)
  text = re.sub(r'[\[\]]', '', text)
  text = re.sub(r'[\n]', '', text)

  text = text.lower()
  return text

df["Text"] = df["Text"].apply(removeNonsense)
df['Text']
# df['Text'].update(pd.Series(data))



0        nature, in the broadest sense, is the natural...
1       "nature" can refer to the phenomena of the phy...
2       the study of nature is a large, if not the onl...
3       although humans are part of nature, human acti...
4        the word nature is borrowed from the old fren...
                              ...                        
7807    qual è stato il tuo errore, ti diamo da mangia...
7808    narcisa ha cambiato i suoi modi in un primo mo...
7809    come' il narcisismo di adesso marian ha detto ...
7810    immagino che non vorrebbe più pane d'oro adess...
7811    terry in realtà assomigli un po 'a quell'angel...
Name: Text, Length: 4655, dtype: object

In [11]:

!python -m spacy download en_core_web_sm
!python -m spacy download fr_core_news_sm
!python -m spacy download it_core_news_sm
!python -m spacy download pt_core_news_sm
!python -m spacy download es_core_news_sm



/usr/local/lib/python3.9/dist-packages/torch/cuda/__init__.py:497: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")
2023-03-19 01:07:31.289048: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/nvidia/lib:/usr/local/nvidia/lib64
2023-03-19 01:07:31.289185: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/nvidia/lib:/usr/local/nvidia/lib64
2023-03-19 01:07:31.289208: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like to use Nvidia GPU with TensorRT, please make sure the missing libraries mentione

***Text Pre-processing***

In [ ]:
nlp_en = spacy.load("en_core_web_sm")
nlp_fr = spacy.load("fr_core_news_sm")
nlp_it = spacy.load("it_core_news_sm")
nlp_pr = spacy.load("pt_core_news_sm")
nlp_sp = spacy.load("es_core_news_sm")

lang_dict = {
    "English": nlp_en,
    "French": nlp_fr,
    "Italian": nlp_it,
    "Portugeese": nlp_pr,
    "Spanish": nlp_sp
    }

def removeNonsense(doc):
  return [token.lemma_ for token in doc if (not token.is_stop) and (not token.is_punct)]

for row in df.iterrows():
  doc = lang_dict[row[1][1]](row[1][0]) 
  print(removeNonsense(doc))

  


***Text Representation, Text -> Vector***

In [14]:
from sklearn.feature_extraction.text import CountVectorizer

count_vectorizer = CountVectorizer()
vect_list = count_vectorizer.fit_transform(df['Text']).toarray()

In [16]:
from sklearn.model_selection import train_test_split

training_x, testing_x, training_y, testing_y = train_test_split(vect_list, y, test_size = 0.40)

In [ ]:
from sklearn.naive_bayes import MultinomialNB

model = MultinomialNB()
model.fit(training_x, training_y)

In [18]:
def predict_language(text):
  x = count_vectorizer.transform([text]).toarray()
  language = model.predict(x)
  language = le.inverse_transform(language)
  print(language[0])